<a href="https://colab.research.google.com/github/marcelleflavya/chatbot_entrelinhas/blob/main/chatbot_entrelinhas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [3]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [5]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [7]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [8]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [13]:
##########################################
# --- Agente 1: Buscador de Livros --- #
##########################################
def agente_buscador_livros(nome_livro):
    buscador = Agent(
        name="agente_buscador_livros",
        model="gemini-2.0-flash",
        instruction="""
        Vocẽ é um assistente de pesquisa e a sua tarefa é usar a ferramenta de busca do google (google_search)
        para pesquisar informações sobre livros a partir do seu título.
        Retorne o nome do livro, o autor e um resumo breve de no maximo 5 linhas sobre do que se trata a historia, focando em detalhes sobre o livro.
        """,
        description="Agente que busca informações no Google sobre livros ",
        tools={google_search},
    )


    entrada_do_agente_buscador_livro= f"Livro: {nome_livro}"

    # Executa o agente
    informacoes_livros= call_agent(buscador, entrada_do_agente_buscador_livro)
    return informacoes_livros

In [14]:
##########################################
# --- Agente 2: Buscador de autores --- #
##########################################
def agente_buscador_autores(nome_autor):
    buscador = Agent(
        name="agente_buscador_autor",
        model="gemini-2.0-flash",
        instruction="""
        Vocẽ é um assistente de pesquisa e a sua tarefa é usar a ferramenta de busca do google (google_search)
        para pesquisar informações sobre um autor a partir do seu nome.
        Retorne o nome do autor e um resumo breve de no maximo 5 linhas sobre a biografia dele, incluindo outros livros que ele tenha publicado.
        """,
        description="Agente que busca informações no Google sobre autores ",
        tools={google_search},
    )

    entrada_do_agente_buscador_autor= f"Autor: {nome_autor}"

    # Executa o agente
    informacoes_livros_autor= call_agent(buscador, entrada_do_agente_buscador_autor)
    return informacoes_livros_autor

In [15]:
##########################################
# --- Agente 3: Informações gerais --- #
##########################################
def agente_informacoes(informacoes_livros ):
    buscador = Agent(
        name="agente_buscador_informacoes",
        model="gemini-2.0-flash",
        instruction="""
        Vocẽ é um assistente de pesquisa e a sua tarefa é usar a ferramenta de busca do google (google_search)
        para pesquisar informações sobre um livro e seu autor, retornando curiosidades interessantes como por exemplo: metodo de escrita,
        momento em que o livro foi desenvolvido, prêmios que o livro possa ter recebido. Reunia as curiosidades e informações mais interessantes em um texto de até 7 linhas.
        """,
        description="Agente que busca informações no Google sobre curiosidades sobre um determinado livro",
        tools={google_search},
    )

    entrada_do_agente_informacoes= f"Livro:{informacoes_livros}"

    # Executa o agente
    informacoes_livros_geral= call_agent(buscador, entrada_do_agente_informacoes)
    return informacoes_livros_geral

In [ ]:
print("Iniciando o Sistema de busca de Livros: ")


print("""-----Escolha uma opção-----\n
1 - Para saber sobre um livro e um breve resumo\n
2 - Para saber sobre um autor e um breve resumo de sua biografia\n
3 - Para saber sobre as curiosidades do seu livro preferido\n
4 - Para sair\n
""")

opcao = input("Digite o número da opção desejada: ")



if opcao == "1":
   nome_livro = input("Olá, digite o nome do livro para buscar informações sobre ele: ")
   print(f'Vamos pesquisar um pouco mais sobre {nome_livro}')
   livro_buscado = agente_buscador_livros(nome_livro)
   print("\n--- Resultado do agente 1 (Buscador de livros) ---\n")
   display(to_markdown(livro_buscado))
   print("-----------------------------------------------------")


elif opcao == "2":
  nome_autor = input("Olá, digite o nome do autor para buscar informações sobre ele: ")
  print(f'Vamos pesquisar um pouco mais sobre {nome_autor}')
  autor_buscado= agente_buscador_autores(nome_autor)
  print("\n--- Resultado do agente 2 (Buscador de autores) ---\n")
  display(to_markdown(autor_buscado))
  print("-----------------------------------------------------")


elif opcao == "3":
        print("Pesquisando curiosidades sobre livros...")
        nome_livro = input("Olá, digite o nome do livro para buscar informações sobre ele: ")
        livro_buscado = agente_buscador_livros(nome_livro)
        informacoes_gerais = agente_informacoes(livro_buscado)
        print("\n--- Resultado do agente 3 (Informações gerais) ---\n")
        display(to_markdown(informacoes_gerais))
        print("-----------------------------------------------------")


elif opcao == "4":
  print("Saindo do sistema...")
  exit()

else:
  print("Opção inválida. Retorne para uma opção válida.")

Iniciando o Sistema de busca de Livros: 
-----Escolha uma opção-----

1 - Para saber sobre um livro e um breve resumo

2 - Para saber sobre um autor e um breve resumo de sua biografia

3 - Para saber sobre as curiosidades do seu livro preferido

4 - Para sair


